In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1032547,2021-02-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1032548,2021-02-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1032549,2021-02-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1032550,2021-02-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28003,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28005,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28007,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28009,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28011,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
639521,2021-02-25,Arkansas,Arkansas,5001,1967,38.00,5000,Arkansas,AR,Arkansas,State,3017804
639523,2021-02-26,Arkansas,Arkansas,5001,1971,38.00,5000,Arkansas,AR,Arkansas,State,3017804
639525,2021-02-27,Arkansas,Arkansas,5001,1980,38.00,5000,Arkansas,AR,Arkansas,State,3017804
639527,2021-02-28,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1032547,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1032548,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1032549,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1032550,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-01') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
405,2021-03-01,Snohomish,Washington,30957,539.00,53061,WA,County,822083,3765.68,65.57
808,2021-03-01,Cook,Illinois,474885,9924.00,17031,IL,County,5150233,9220.65,192.69
1210,2021-03-01,Orange,California,261408,3921.00,6059,CA,County,3175692,8231.53,123.47
1611,2021-03-01,Maricopa,Arizona,511055,9116.00,4013,AZ,County,4485414,11393.71,203.24
2012,2021-03-01,Los Angeles,California,1193531,21467.00,6037,CA,County,10039107,11888.82,213.83
...,...,...,...,...,...,...,...,...,...,...,...
1032115,2021-03-01,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1032255,2021-03-01,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1032364,2021-03-01,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1032469,2021-03-01,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
405,2021-03-01,Snohomish,Washington,30957,539.00,53061,WA,County,822083,3765.68,65.57,65.57,3765.68
808,2021-03-01,Cook,Illinois,474885,9924.00,17031,IL,County,5150233,9220.65,192.69,192.69,9220.65
1210,2021-03-01,Orange,California,261408,3921.00,6059,CA,County,3175692,8231.53,123.47,123.47,8231.53
1611,2021-03-01,Maricopa,Arizona,511055,9116.00,4013,AZ,County,4485414,11393.71,203.24,203.24,11393.71
2012,2021-03-01,Los Angeles,California,1193531,21467.00,6037,CA,County,10039107,11888.82,213.83,213.83,11888.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032115,2021-03-01,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1032255,2021-03-01,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1032364,2021-03-01,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1032469,2021-03-01,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
727489,2021-03-01,Clarke,Alabama,3436,50.00,1025,AL,County,23622,14545.76,211.67,211.67,14545.76,1
668594,2021-03-01,Hale,Alabama,2105,68.00,1065,AL,County,14651,14367.62,464.13,464.13,14367.62,2
523581,2021-03-01,Lowndes,Alabama,1311,51.00,1085,AL,County,9726,13479.33,524.37,524.37,13479.33,3
464421,2021-03-01,Etowah,Alabama,13184,319.00,1055,AL,County,102268,12891.62,311.93,311.93,12891.62,4
389916,2021-03-01,Franklin,Alabama,3991,77.00,1059,AL,County,31362,12725.59,245.52,245.52,12725.59,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638176,2021-03-01,Converse,Wyoming,970,17.00,56009,WY,County,13822,7017.80,122.99,122.99,7017.80,19
638515,2021-03-01,Sublette,Wyoming,676,7.00,56035,WY,County,9831,6876.21,71.20,71.20,6876.21,20
881537,2021-03-01,Niobrara,Wyoming,149,2.00,56027,WY,County,2356,6324.28,84.89,84.89,6324.28,21
792621,2021-03-01,Lincoln,Wyoming,1235,11.00,56023,WY,County,19830,6227.94,55.47,55.47,6227.94,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
523581,2021-03-01,Lowndes,Alabama,1311,51.00,1085,AL,County,9726,13479.33,524.37,524.37,13479.33,3,1
668594,2021-03-01,Hale,Alabama,2105,68.00,1065,AL,County,14651,14367.62,464.13,464.13,14367.62,2,2
463053,2021-03-01,Clay,Alabama,1444,54.00,1027,AL,County,13235,10910.46,408.01,408.01,10910.46,21,3
223210,2021-03-01,Walker,Alabama,6524,255.00,1127,AL,County,63521,10270.62,401.44,401.44,10270.62,32,4
564498,2021-03-01,Greene,Alabama,882,32.00,1063,AL,County,8111,10874.12,394.53,394.53,10874.12,22,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563137,2021-03-01,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
772974,2021-03-01,Uinta,Wyoming,2055,12.00,56041,WY,County,20226,10160.19,59.33,59.33,10160.19,4,20
792621,2021-03-01,Lincoln,Wyoming,1235,11.00,56023,WY,County,19830,6227.94,55.47,55.47,6227.94,22,21
222166,2021-03-01,Teton,Wyoming,3357,9.00,56039,WY,County,23464,14307.02,38.36,38.36,14307.02,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.57,3765.68,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.57,3765.68,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.57,3765.68,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.57,3765.68,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.57,3765.68,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030111,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1030112,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1030113,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1030114,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
724719,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,211.67,14545.76,46,1,2.00
724720,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,211.67,14545.76,46,1,0.00
724721,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,211.67,14545.76,46,1,1.00
724722,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,211.67,14545.76,46,1,4.00
724723,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,211.67,14545.76,46,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805785,2021-02-25,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
805786,2021-02-26,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
805787,2021-02-27,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
805788,2021-02-28,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
521483,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13479.33,1,3,1.00,0.00
521484,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13479.33,1,3,0.00,0.00
521485,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13479.33,1,3,0.00,0.00
521486,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13479.33,1,3,0.00,0.00
521487,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13479.33,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519773,2021-02-25,Albany,Wyoming,3894,11.00,56001,WY,County,38880,10015.43,28.29,28.29,10054.01,23,6,1.00,0.00
519774,2021-02-26,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10054.01,23,6,7.00,0.00
519775,2021-02-27,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10054.01,23,6,0.00,0.00
519776,2021-02-28,Albany,Wyoming,3905,11.00,56001,WY,County,38880,10043.72,28.29,28.29,10054.01,23,6,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-01') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
266330,2021-03-01,Imperial,California,26909,632.00,6025,CA,County,181215,14849.21,348.76,348.76,14849.21,1,2,6.00,42.00
2012,2021-03-01,Los Angeles,California,1193531,21467.00,6037,CA,County,10039107,11888.82,213.83,213.83,11888.82,2,5,972.00,32.00
465741,2021-03-01,Inyo,California,1282,37.00,6027,CA,County,18039,7106.82,205.11,205.11,7106.82,3,27,7.00,0.00
59521,2021-03-01,Stanislaus,California,56235,949.00,6099,CA,County,550660,10212.29,172.34,172.34,10212.29,4,10,103.00,3.00
73758,2021-03-01,Tulare,California,48013,761.00,6107,CA,County,466195,10298.91,163.24,163.24,10298.91,5,9,144.00,3.00
27642,2021-03-01,Riverside,California,289773,3792.00,6065,CA,County,2470546,11729.11,153.49,153.49,11729.11,6,6,323.00,25.00
47385,2021-03-01,San Joaquin,California,66829,1126.00,6077,CA,County,762148,8768.51,147.74,147.74,8768.51,7,18,120.00,25.00
26922,2021-03-01,Fresno,California,95548,1443.00,6019,CA,County,999101,9563.40,144.43,144.43,9563.40,8,14,155.00,0.00
422197,2021-03-01,Merced,California,29147,401.00,6047,CA,County,277680,10496.61,144.41,144.41,10496.61,9,8,176.00,4.00
564440,2021-03-01,Kings,California,22082,220.00,6031,CA,County,152940,14438.34,143.85,143.85,14438.34,10,3,22.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
981295,2021-03-01,Lassen,California,5601,19.00,6035,CA,County,30573,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
266330,2021-03-01,Imperial,California,26909,632.00,6025,CA,County,181215,14849.21,348.76,348.76,14849.21,1,2,6.00,42.00
564440,2021-03-01,Kings,California,22082,220.00,6031,CA,County,152940,14438.34,143.85,143.85,14438.34,10,3,22.00,0.00
132852,2021-03-01,San Bernardino,California,286755,2940.00,6071,CA,County,2180085,13153.39,134.86,134.86,13153.39,12,4,148.00,66.00
2012,2021-03-01,Los Angeles,California,1193531,21467.00,6037,CA,County,10039107,11888.82,213.83,213.83,11888.82,2,5,972.00,32.00
27642,2021-03-01,Riverside,California,289773,3792.00,6065,CA,County,2470546,11729.11,153.49,153.49,11729.11,6,6,323.00,25.00
166941,2021-03-01,Kern,California,103422,877.00,6029,CA,County,900202,11488.75,97.42,97.42,11488.75,20,7,127.00,26.00
422197,2021-03-01,Merced,California,29147,401.00,6047,CA,County,277680,10496.61,144.41,144.41,10496.61,9,8,176.00,4.00
73758,2021-03-01,Tulare,California,48013,761.00,6107,CA,County,466195,10298.91,163.24,163.24,10298.91,5,9,144.00,3.00
59521,2021-03-01,Stanislaus,California,56235,949.00,6099,CA,County,550660,10212.29,172.34,172.34,10212.29,4,10,103.00,3.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
266330,2021-03-01,Imperial,California,26909,632.00,6025,CA,County,181215,14849.21,348.76,348.76,14849.21,1,2,6.00,42.00
2012,2021-03-01,Los Angeles,California,1193531,21467.00,6037,CA,County,10039107,11888.82,213.83,213.83,11888.82,2,5,972.00,32.00
465741,2021-03-01,Inyo,California,1282,37.00,6027,CA,County,18039,7106.82,205.11,205.11,7106.82,3,27,7.00,0.00
59521,2021-03-01,Stanislaus,California,56235,949.00,6099,CA,County,550660,10212.29,172.34,172.34,10212.29,4,10,103.00,3.00
73758,2021-03-01,Tulare,California,48013,761.00,6107,CA,County,466195,10298.91,163.24,163.24,10298.91,5,9,144.00,3.00
27642,2021-03-01,Riverside,California,289773,3792.00,6065,CA,County,2470546,11729.11,153.49,153.49,11729.11,6,6,323.00,25.00
47385,2021-03-01,San Joaquin,California,66829,1126.00,6077,CA,County,762148,8768.51,147.74,147.74,8768.51,7,18,120.00,25.00
26922,2021-03-01,Fresno,California,95548,1443.00,6019,CA,County,999101,9563.40,144.43,144.43,9563.40,8,14,155.00,0.00
422197,2021-03-01,Merced,California,29147,401.00,6047,CA,County,277680,10496.61,144.41,144.41,10496.61,9,8,176.00,4.00
564440,2021-03-01,Kings,California,22082,220.00,6031,CA,County,152940,14438.34,143.85,143.85,14438.34,10,3,22.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,348.76,14849.21,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3482,02/25/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,7.00,0.00
3483,02/26/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
3484,02/27/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
3485,02/28/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,348.76,14849.21,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,348.76,14849.21,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3556,02/25/21,Kings,21951,218.00,14352.69,142.54,143.85,14438.34,10,3,35.00,0.00
3557,02/26/21,Kings,22003,218.00,14386.69,142.54,143.85,14438.34,10,3,52.00,0.00
3558,02/27/21,Kings,22042,218.00,14412.19,142.54,143.85,14438.34,10,3,39.00,0.00
3559,02/28/21,Kings,22060,220.00,14423.96,143.85,143.85,14438.34,10,3,18.00,2.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)